In [1]:
pip install xgboost lightgbm catboost optuna

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 2.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 27.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns

from math import floor

from scipy.stats import rankdata
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

import optuna as op

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/Tabular-Playground-Jan-2023-4/train.csv'
file_key_2 = 'Tabular-Playground-Series/Tabular-Playground-Jan-2023-4/test.csv'
file_key_3 = 'Tabular-Playground-Series/Tabular-Playground-Jan-2023-4/sample_submission.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

## Reading data files
train = pd.read_csv(file_content_stream_1)
test = pd.read_csv(file_content_stream_2)
submission = pd.read_csv(file_content_stream_3)

train['Time'] = train['Time'] / 3600
test['Time'] = test['Time'] / 3600

train = train[(train['Time'] >= 10) & (train['Time'] <= 24)].reset_index(drop = True)
train['Time_Bucket'] = train['Time'].apply(floor)

# train['Time_Bucket'] = train['Time_Bucket'] - 8

# test['Time_Bucket'] = test['Time'].apply(floor)
# test['Time_Bucket'] = test['Time_Bucket'] - 32

In [5]:
train.head()

,id,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class,Time_Bucket
0,28639,10.0025,1.240257,-0.064968,0.292301,0.562824,-0.463260,-0.643769,-0.290874,0.074817,0.520861,-0.059493,-0.401788,-0.906394,-2.191959,0.763318,1.537556,0.055868,-0.018766,-0.812256,-0.466929,-0.270077,-0.269914,-0.804726,0.184878,-0.017442,0.066157,0.139930,-0.032306,0.005789,1.98,0,10
1,28640,10.0025,0.566687,-1.065377,0.843466,1.347078,-1.132464,0.499793,-0.466450,0.325933,0.565898,-0.102979,1.255589,0.441395,-1.050434,0.354097,1.148959,0.492468,-0.520868,0.638387,-0.719238,0.420869,0.390013,0.530061,-0.381049,0.027772,0.228536,-0.263094,0.024399,0.072729,295.20,0,10
2,28641,10.0025,-0.666978,0.582365,1.320170,1.038262,-0.717517,-0.698978,-0.358901,0.562265,0.418491,-0.639646,-0.848472,-0.099086,-1.336930,0.183388,-0.006867,-0.326828,0.329010,-0.116537,0.443480,-0.230117,-0.022416,-0.054777,0.161706,0.737483,-1.037992,-0.385097,0.128122,0.143395,5.75,0,10
3,28642,10.0025,0.981675,-0.207519,1.261230,1.510517,-1.050243,-0.001808,-0.492566,0.126880,0.865598,-0.261810,-0.594468,0.624652,0.113990,-0.402636,0.186490,-0.016023,-0.056938,-0.543408,-0.397957,-0.025256,-0.037674,0.013838,-0.038693,0.443485,0.383349,-0.429776,0.064349,0.051706,74.99,0,10
4,28643,10.0025,1.152601,0.181725,1.235559,1.073024,-0.812595,-0.337824,-0.438645,0.043215,0.373339,-0.047564,-0.230448,0.547962,0.602038,-0.065167,1.427969,0.368845,-0.505403,-0.160750,-0.886036,-0.112488,0.044444,0.205976,0.062316,0.390864,0.270659,-0.421760,0.065232,0.036674,0.12,0,10


In [4]:
train['Time_Bucket'].value_counts()

20    15388
17    15016
14    14878
21    14346
22    13707
19    12896
15    12773
18    12295
16    11797
13    10201
10     9258
12     8452
23     8394
11     4903
24        2
Name: Time_Bucket, dtype: int64

In [82]:
train[train['Time_Bucket'] == 24]

,id,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class,Time_Bucket
164304,192943,24.0,-0.332390,2.140909,-1.717079,1.154868,1.076970,-1.716594,1.471908,-0.867357,0.452747,-0.639773,0.652949,-0.940592,-0.557781,-2.814991,1.057944,0.113925,1.867007,1.252300,-0.248925,0.098387,0.052959,0.570145,-0.286617,0.001668,-0.525930,-0.410837,-0.839087,-0.286535,1.00,0,24
164305,192944,24.0,1.889587,-0.009617,-2.724135,0.619191,0.734012,-1.107895,0.808830,-0.455867,0.382389,-0.711441,-0.869179,0.094339,0.276679,-1.175027,-0.184964,0.258653,0.818112,-0.139536,0.194766,0.053351,-0.322571,-0.946699,-0.068503,-0.814458,-0.205539,-0.058140,-0.118637,-0.019226,129.95,0,24


In [86]:
test['Time_Bucket'].value_counts()

42    13980
39    13420
34    12596
40    11550
45    11393
37    10857
36    10465
44     9787
38     9459
43     9188
41     8235
35     8112
33     6974
46     6017
47     4054
Name: Time_Bucket, dtype: int64

In [39]:
train.head()

,id,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,22365,9.000556,1.244814,-0.754958,0.854670,-0.735576,-1.332617,-0.308208,-1.017979,0.162837,-0.715797,0.754855,1.630481,-0.222389,-0.771056,0.082871,0.568529,1.273936,0.183846,-1.035992,0.146824,0.022408,0.249969,0.580945,0.028481,0.223534,0.180780,-0.255845,0.023028,0.012719,29.95,0
1,22366,9.001111,1.267944,-0.834604,0.972069,-0.470734,-1.106470,0.331408,-0.971756,0.183514,-0.579641,0.477559,1.547337,1.082639,0.926375,-0.653915,-0.977970,0.366421,0.808828,-1.935260,0.706095,0.083736,-0.065784,0.475079,0.027949,0.282710,0.336041,-0.221849,0.060442,0.004089,10.00,0
2,22367,9.004167,1.285540,-0.919086,0.715597,-0.591645,-0.945174,1.005579,-1.329266,0.282678,0.302076,0.250115,-2.023751,-0.402224,0.130148,-1.027361,-1.567678,0.264579,0.892230,-1.567146,1.333612,0.045291,0.174450,0.882723,-0.251101,-0.889915,0.767446,0.127534,0.078694,-0.000612,1.00,0
3,22368,9.004722,1.349206,-0.865703,1.240889,-0.681399,-1.779078,-0.352529,-1.322768,0.068639,0.006841,0.569909,-0.816478,-0.937795,-0.427934,-0.569345,0.501294,1.398905,0.190789,-0.781538,0.469772,0.108236,0.358264,1.021212,-0.089879,0.404026,0.382348,-0.052998,0.051717,0.027174,9.99,0
4,22369,9.005000,-0.689537,-0.156537,2.557515,-1.788165,-0.881340,0.964190,-0.808502,0.646445,-0.818292,0.047035,1.447565,-0.738241,-1.040988,-0.323057,0.929153,1.164949,0.366433,-0.861169,-0.571727,0.056119,0.592060,1.604599,-0.229814,-0.350044,-0.128974,-0.057114,0.084733,0.074817,1.00,0


In [55]:
train['Time'].describe()

count    164306.000000
mean         17.496895
std           3.730481
min          10.002500
25%          14.574167
50%          17.638333
75%          20.675833
max          24.000000
Name: Time, dtype: float64

In [52]:
test['Time'].describe()

count    146087.000000
mean         40.177202
std           3.960563
min          33.494444
25%          36.860556
50%          40.136944
75%          43.372222
max          47.997222
Name: Time, dtype: float64

In [42]:
train['Time_Bucket'] = train['Time'].apply(floor)
train = train[train['Time_Bucket'] < 24].reset_index(drop = True)
train['Time_Bucket'] = train['Time_Bucket'] - 8

test['Time_Bucket'] = test['Time'].apply(floor)
test['Time_Bucket'] = test['Time_Bucket'] - 32

train.head()

,id,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class,Time_Bucket
0,22365,9.000556,1.244814,-0.754958,0.854670,-0.735576,-1.332617,-0.308208,-1.017979,0.162837,-0.715797,0.754855,1.630481,-0.222389,-0.771056,0.082871,0.568529,1.273936,0.183846,-1.035992,0.146824,0.022408,0.249969,0.580945,0.028481,0.223534,0.180780,-0.255845,0.023028,0.012719,29.95,0,1
1,22366,9.001111,1.267944,-0.834604,0.972069,-0.470734,-1.106470,0.331408,-0.971756,0.183514,-0.579641,0.477559,1.547337,1.082639,0.926375,-0.653915,-0.977970,0.366421,0.808828,-1.935260,0.706095,0.083736,-0.065784,0.475079,0.027949,0.282710,0.336041,-0.221849,0.060442,0.004089,10.00,0,1
2,22367,9.004167,1.285540,-0.919086,0.715597,-0.591645,-0.945174,1.005579,-1.329266,0.282678,0.302076,0.250115,-2.023751,-0.402224,0.130148,-1.027361,-1.567678,0.264579,0.892230,-1.567146,1.333612,0.045291,0.174450,0.882723,-0.251101,-0.889915,0.767446,0.127534,0.078694,-0.000612,1.00,0,1
3,22368,9.004722,1.349206,-0.865703,1.240889,-0.681399,-1.779078,-0.352529,-1.322768,0.068639,0.006841,0.569909,-0.816478,-0.937795,-0.427934,-0.569345,0.501294,1.398905,0.190789,-0.781538,0.469772,0.108236,0.358264,1.021212,-0.089879,0.404026,0.382348,-0.052998,0.051717,0.027174,9.99,0,1
4,22369,9.005000,-0.689537,-0.156537,2.557515,-1.788165,-0.881340,0.964190,-0.808502,0.646445,-0.818292,0.047035,1.447565,-0.738241,-1.040988,-0.323057,0.929153,1.164949,0.366433,-0.861169,-0.571727,0.056119,0.592060,1.604599,-0.229814,-0.350044,-0.128974,-0.057114,0.084733,0.074817,1.00,0,1


In [43]:
train['Time_Bucket'].value_counts()

12    15388
9     15016
6     14878
13    14346
14    13707
11    12896
7     12773
10    12295
8     11797
5     10201
2      9258
4      8452
15     8394
1      6274
3      4903
Name: Time_Bucket, dtype: int64

In [77]:
train.head()

,id,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,22365,9.000556,1.244814,-0.754958,0.854670,-0.735576,-1.332617,-0.308208,-1.017979,0.162837,-0.715797,0.754855,1.630481,-0.222389,-0.771056,0.082871,0.568529,1.273936,0.183846,-1.035992,0.146824,0.022408,0.249969,0.580945,0.028481,0.223534,0.180780,-0.255845,0.023028,0.012719,29.95,0
1,22366,9.001111,1.267944,-0.834604,0.972069,-0.470734,-1.106470,0.331408,-0.971756,0.183514,-0.579641,0.477559,1.547337,1.082639,0.926375,-0.653915,-0.977970,0.366421,0.808828,-1.935260,0.706095,0.083736,-0.065784,0.475079,0.027949,0.282710,0.336041,-0.221849,0.060442,0.004089,10.00,0
2,22367,9.004167,1.285540,-0.919086,0.715597,-0.591645,-0.945174,1.005579,-1.329266,0.282678,0.302076,0.250115,-2.023751,-0.402224,0.130148,-1.027361,-1.567678,0.264579,0.892230,-1.567146,1.333612,0.045291,0.174450,0.882723,-0.251101,-0.889915,0.767446,0.127534,0.078694,-0.000612,1.00,0
3,22368,9.004722,1.349206,-0.865703,1.240889,-0.681399,-1.779078,-0.352529,-1.322768,0.068639,0.006841,0.569909,-0.816478,-0.937795,-0.427934,-0.569345,0.501294,1.398905,0.190789,-0.781538,0.469772,0.108236,0.358264,1.021212,-0.089879,0.404026,0.382348,-0.052998,0.051717,0.027174,9.99,0
4,22369,9.005000,-0.689537,-0.156537,2.557515,-1.788165,-0.881340,0.964190,-0.808502,0.646445,-0.818292,0.047035,1.447565,-0.738241,-1.040988,-0.323057,0.929153,1.164949,0.366433,-0.861169,-0.571727,0.056119,0.592060,1.604599,-0.229814,-0.350044,-0.128974,-0.057114,0.084733,0.074817,1.00,0


# XGBoost Modeling

In [22]:
X = train.drop(columns = ['id', 'Time', 'Class'], axis = 1)
Y = train['Class']
# test = test.drop(columns = ['id', 'Time'], axis = 1)

XGB_cv_scores, XGB_imp = list(), list()
preds = list()

## Running 5 times CV
for i in range(5):
    
    skf = StratifiedKFold(n_splits = 5, shuffle = False)
    
    for train_ix, test_ix in skf.split(X, Y):
        
        ## Splitting the data 
        X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
        Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
                
        ## Building RF model
        XGB_md = XGBClassifier(tree_method = 'hist',
                               colsample_bytree = 0.7, 
                               gamma = 0.72, 
                               learning_rate = 0.01, 
                               max_depth = 9, 
                               min_child_weight = 10, 
                               n_estimators = 1000, 
                               subsample = 0.7).fit(X_train, Y_train)
        XGB_imp.append(XGB_md.feature_importances_)
        
        ## Predicting on X_test and test
        XGB_pred_1 = XGB_md.predict_proba(X_test)[:, 1]
        XGB_pred_2 = XGB_md.predict_proba(test)[:, 1]
        
        ## Computing roc-auc score
        XGB_cv_scores.append(roc_auc_score(Y_test, XGB_pred_1))
        preds.append(XGB_pred_2)

XGB_cv_score = np.mean(XGB_cv_scores)    
print('The average roc-auc score over 5-folds (run 5 times) is:', XGB_cv_score)

The average roc-auc score over 5-folds (run 5 times) is: 0.8206248860330629


In [18]:
XGB_preds_test = pd.DataFrame(preds).apply(np.mean, axis = 0)
submission['Class'] = XGB_preds_test
submission.to_csv('xgb_submission_8.csv', index = False)